In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'nithan-chadok-name-entity-recognition:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F71609%2F7833551%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240308%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240308T125849Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D07bf986af011e32bb07c08454b9d6232288f08d1eff662b7498d5eb8458061df2bfd0707456e3b6c5d803f7829c27a93028aa9cd36d121de698342cf018a63496dd4a86d5d24c32c6f11834329c69a6407607c34726720e2155a7a62279d73143b5c41d1ac3c155c874b6ca2248588442f0bea9682c2ddd630f170d4e2c795a15e362ed3a56fcddfd11091508a3a9a7680ce1b0b734114794e75e45a57e59f6204ee9265e59421b34f662db0f708f79d7b74d016756902fc6e5a7433a72d2b0c538809d4563eb2e03d4aa1f626739885176adb78794db331f3c244888df70f262128fc7f960abfe987ea658a701e6d815cc6c1015610383f181011ffebf3dd1e,lst20-thai:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4521401%2F7736495%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240308%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240308T125849Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2afc4a24b0850021e463c05f3c271d57d2840789deeea373d4f45dbecde2c2b009a0800bb55c30767a743308ea7473cc3b315bfbd692a41bd434b22bb59f11db377c4f4caed21ea7c3c2a5677b54ada7f60300115cdc6aa6b016787aa0f207c2a59c0fba13e6e7dbc3e7be0c3082f47b3af8a34890f40d20c5b86d33a9073a6ee07fe709c2aa0a2891f3ac0bb288227c4e0c68116f7b29c58271bd67af35d4572d68e6366e0ee8f202a05562e8c923780a95b56390176494f03cba3792149812080ab8c5cdf0f30bbe1c75434b312e24fddd0b856950f3548968e7eeecdd005693c19a1ae941f38017153fa8ff9987c6641f2e65f990c76483810eca4c37d07e,thanapat-berta-15:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4526188%2F7743359%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240308%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240308T125849Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D0586d22f7dfa435937395c5a05da0cdaffdbca729c93e959ba5796f17c1b8317c55596c43f11d8f6e2f31dc172d5e3b7bc507f20c135cf8fdfdc4df459c2b8017a228087c85031d9d4b1c3db7013f562c504e3e02b07a356355ca98c42c8c711fa840dac5fc2834db42e7db9d57478486500378eb2b0199a05f0ae66d88afab4ada242b9c192e83a8e73037eb8f54c849b41c0d54a884e8551cec56052220b109b8a97469dbc18ea00f20ebd5741d1f5fe64c54080f6a2503117937bc9dbc836356b42210c1db516403c4562d72e2aaae83293442de0eff61146aad848078eaaf8a57f76e7460a6eb4426d5affbf8d6606029a437c72fcb4fa37235eaa933634,roberta-xml:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4527290%2F7744876%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240308%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240308T125849Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D584fe6b7522d022446ced4334f639e8127a6728a88cc71a5744f6b23d2a389cb34645aa8accbe6f09629c8772c580d89568f973be6497c3e79937c46ab9830d5e2a4b2439ea465db49522c9e635c9983afc8025f85363eb6df9d710dfa2005363ef21efea30c5220afb95743b728c04550b1109fd4add0f890a6df202011bb93ecb8f53ac2c2ae30c0911c6c97f1879926c3bda4dfa9be91369d9d1e493939d4b15c0dc2149430f0f9715804f9838d549cea58c75b92ac3d8e3643bb3b412f9c4ef3b0e720f5b26c95fb67bb8aa94deacdd92db731c7d73034948d8b6ff16ab834833e752df6fea4235bf15cabac3e5dffd477b38d6332bf4aa134312319a3c5,thanapat-wang225:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4527403%2F7745033%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240308%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240308T125849Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D5d347de723a39cdab02a8841f0e1a07cb948c8f4d3ed81db5161c2427adf70d968382976795d74776f798dd911d009b1b98d491ac8ef938521aa73e5e25a72a84ee695baa0f88d52a5e6a814ee97c9cfc75745c9c96b4ce2f9fea734f2f5bed5d392750b9120a0bf21b4d458c528b90d2b7cf2d4a7e662b922a4084578fe24b09fe547226dc2a4ac0ee6079ddfd8f5882ebb3ad61c9cd8838a1a6da836d95796921767482da638d88c4be30c597d59a896f7cb9b59150201ca59ef626ee2d692e896af0d8574a7079d23fe58b7a06d9f1225a4ed4eb689d955762987c6bb64c6ef71eefa2ba86c6d247aa7f68e4d897fc2679a027a09927e3aff0deceb277497'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install -q simpletransformers

In [ ]:
import numpy as np
import pandas as pd
import json
from datasets import load_dataset

# Load dataset to DataFrame

In [ ]:
#Load file in directory into variable lst20
lst20 = load_dataset("lst20", data_dir="/kaggle/input/lst20-thai/LST20_Corpus")
lst20

In [ ]:
#Convert to dataFrame
train_df = pd.DataFrame(lst20['train'])
validation_df = pd.DataFrame(lst20['validation'])
test_df = pd.DataFrame(lst20['test'])
train_df.head(3)

In [ ]:
train_df = train_df[['id', 'tokens', 'ner_tags']]
validation_df = validation_df[['id', 'tokens', 'ner_tags']]
test_df = test_df[['id', 'tokens', 'ner_tags']]
train_df

In [ ]:
# # Sorting NER Tags start from O, B, I, and E
# ner_list = [item.strip().strip(" '") for item in ner_list.strip("[]\n").split(",")]
# O_list = [item for item in ner_list if item.startswith("O")]
# B_list = [item for item in ner_list if item.startswith("B_")]
# I_list = [item for item in ner_list if item.startswith("I_")]
# E_list = [item for item in ner_list if item.startswith("E_")]
# #Sort start from O,B_,I_,E_
# NER_TAGS = O_list + B_list + I_list + E_list
# print(NER_TAGS)

In [ ]:
# Define Ner Tags replace missing ne_list.txt
NER_TAGS = [
       "O",
        "B_BRN", "B_DES", "B_DTM", "B_LOC", "B_MEA", "B_NUM", "B_ORG", "B_PER", "B_TRM", "B_TTL",
       "I_BRN", "I_DES", "I_DTM", "I_LOC", "I_MEA", "I_NUM", "I_ORG", "I_PER", "I_TRM", "I_TTL",
        "E_BRN", "E_DES", "E_DTM",  "E_LOC", "E_MEA", "E_NUM", "E_ORG", "E_PER", "E_TRM", "E_TTL",
        "B_NAME"]
print(NER_TAGS)

# Simpletransformers Data Formats

In [ ]:
#Code NER Tags mapping
#[sentenceid][token]
print(train_df['ner_tags'][0][0])
print(NER_TAGS[train_df['ner_tags'][0][0]])

In [ ]:
#Simple Transformer format
#Append all word into the same dataFrame
#Extract Data form Data Loader
def convert_data_to_df(df):
  data_df = pd.DataFrame()
  sentence_id = []
  words = []
  labels = []

  for sentence in range(len(df)):
    for token in range(len(df['tokens'][sentence])):
      sentence_id.append(sentence)
      words.append(df['tokens'][sentence][token])
      labels.append(NER_TAGS[df['ner_tags'][sentence][token]]) #Map 0 to "O", 1 to "B_BRN"

  return pd.DataFrame(
      {"sentence_id": sentence_id, "words": words, "labels": labels}
  )

In [ ]:
train_df.head()

In [ ]:
train_data = convert_data_to_df(train_df)
#Re-process to validate and test dataset
eval_data = convert_data_to_df(validation_df )
test_data = convert_data_to_df(test_df)
train_data.head(10)

In [ ]:
import logging
from simpletransformers.ner import NERModel, NERArgs
import torch

# Simple Transformer https://simpletransformers.ai/docs/ner-minimal-start/
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Config NER Model

In [ ]:
max_seq_length = train_data['words'].str.len().max()
print("Maximum length in column 'words':", max_seq_length)

In [ ]:
# Configure the model
# ner_args = NERArgs()
# ner_args.train_batch_size = 16 #192 is fit for GPU T4, 512 for A100
# ner_args.use_multiprocessing = True
# ner_args.evaluate_during_training = True
# ner_args.eval_batch_size = 16
# ner_args.num_train_epochs = 5
# ner_args.overwrite_output_dir = True
# ner_args.learning_rate = 1e-4

# model = NERModel(
#      "camembert", # Model Type
#      "airesearch/wangchanberta-base-att-spm-uncased",  #Ner Pre-trained Model
#      args=ner_args, use_cuda=torch.cuda.is_available(), labels=NER_TAGS # Local Config
# )

In [ ]:
ner_args = NERArgs()
ner_args.train_batch_size = 128
ner_args.use_multiprocessing = True
ner_args.evaluate_during_training = True
ner_args.eval_batch_size = 128
ner_args.num_train_epochs = 10
ner_args.overwrite_output_dir = True
ner_args.learning_rate = 1e-4

model = NERModel(
     "roberta",
     "xlm-roberta-base",
     args=ner_args, use_cuda=torch.cuda.is_available(), labels=NER_TAGS
)

In [ ]:
import torch
torch.cuda.empty_cache()
!rm -r /kaggle/working/*

In [ ]:
# train nannnn makmakkk
# model.train_model(train_data, eval_data=eval_data)

In [ ]:
# Evaluate the model
# result, model_outputs, preds_list = model.eval_model(eval_data)

In [ ]:
# print(result)

## Save best model

In [ ]:
#Copy best model folderinto gdrive folder
# !cp -r /content/outputs/best_model /content/drive/MyDrive/colab/hack4/best_model #your destination

# Load Data Test

In [ ]:
submit_df = pd.read_csv('/kaggle/input/nithan-chadok-name-entity-recognition/sample_submission.csv')
test_df = pd.read_csv('/kaggle/input/nithan-chadok-name-entity-recognition/test.csv')
test_df.head(20)

In [ ]:
NER_TAGS = [
       "O",
        "B_BRN","B_DES","B_DTM","B_LOC","B_MEA","B_NUM","B_ORG","B_PER","B_TRM","B_TTL",
       "I_BRN","I_DES","I_DTM","I_LOC","I_MEA","I_NUM","I_ORG","I_PER","I_TRM","I_TTL",
        "E_BRN","E_DES","E_DTM","E_LOC","E_MEA","E_NUM","E_ORG","E_PER","E_TRM","E_TTL"]
len(NER_TAGS)

In [ ]:
txt = []
for i in test_df['word']:
    txt.append(i)

In [ ]:
txt[:20]

In [ ]:
# Sentence Tokenizer every 20 words
def split_into_sentences(tokens, tokens_per_sentence=50):
    sentences = []
    for i in range(0, len(tokens), tokens_per_sentence):
        sentence = tokens[i:i+tokens_per_sentence]
        sentences.append(sentence)
    return sentences

# try 15 > 225 > 50

In [ ]:
my_token = split_into_sentences(txt)
len(my_token)

# Prediction by WangchanBERTa-LST20

In [ ]:
# Test Model
import torch
from simpletransformers.ner import NERModel, NERArgs

ner_args = NERArgs()
ner_args.eval_batch_size = 8
ner_args.use_multiprocessing = True
ner_args.max_seq_length = 510 # Fixed Requirement ##############################
model = NERModel(
     "auto", "thanaphatt1/WangchanBERTa-LST20", args=ner_args, use_cuda=torch.cuda.is_available(), labels= NER_TAGS  # your latest model
)

In [ ]:
# Make predictions with the model
predictions, raw_outputs = model.predict(my_token, False)

In [ ]:
#Check result sentence
len(predictions)

In [ ]:
#Check result data and type
predictions[0]

In [ ]:
tag_df = pd.read_csv('/kaggle/input/nithan-chadok-name-entity-recognition/tag_list.csv')
tag_df

In [ ]:
#Extract data value from dict list
final_test_df = []

for i in predictions:
    for j in i:
        for k in j.values():
            result = tag_df[tag_df['tag'] == k]['class'].values[0]
            final_test_df.append(result)

In [ ]:
#print result
print(len(final_test_df))

In [ ]:
final_result = pd.DataFrame(final_test_df)
submit_df['pred'] = final_result
submit_df

In [ ]:
submit_df[submit_df['pred'].isnull()==True]

In [ ]:
thanapat_15 = pd.read_csv('/kaggle/input/thanapat-berta-15/wangchan_berta_15.csv')
compare = pd.concat([submit_df['i'], submit_df['pred'], thanapat_15['pred']], axis=1)
compare.columns=['i','50','15']
compare[compare['50'] != compare['15']]

In [ ]:
#Save result file to CSV
submit_df.to_csv('thanapatt_wangchan_50.csv',index=False)

## ensemble

In [ ]:
thanapat_15 = pd.read_csv('/kaggle/input/thanapat-berta-15/wangchan_berta_15.csv')
thanapat_225 = pd.read_csv('/kaggle/input/thanapat-wang225/thanapatt_wangchan_225.csv')
# thanapat_50 = pd.read_csv('/kaggle/working/thanapatt_wangchan_50.csv')
robert_boss = pd.read_csv('/kaggle/input/roberta-xml/RobertaXML_1e5_epochs10_split254.csv')

In [ ]:
df_ensemble = pd.concat([thanapat_15,thanapat_225,robert_boss])
df_ensemble = df_ensemble.groupby("i")["pred"].apply(lambda x: x.mode()[0]).reset_index()
df_ensemble

In [ ]:
compare = pd.concat([df_ensemble['i'], df_ensemble['pred'], thanapat_15['pred'], robert_boss['pred']], axis=1)
compare.columns=['i','ens','th_15','boss']
compare[compare['ens'] != compare['th_15']]

In [ ]:
compare[compare['ens'] != compare['boss']]

In [ ]:
df_ensemble.to_csv('ensemble_15_255_boss.csv',index=False)

# Prediction by airesearch_wangchan

In [ ]:
# Test Model
ner_args = NERArgs()
ner_args.eval_batch_size = 8
ner_args.use_multiprocessing = True
# ner_args.max_seq_length = 500 # Fixed Requirement ##############################
model = NERModel(
     "camembert", "airesearch/wangchanberta-base-att-spm-uncased", args=ner_args, use_cuda=torch.cuda.is_available(), labels= NER_TAGS  # your latest model
)

In [ ]:
# Make predictions with the model
predictions, raw_outputs = model.predict(my_token, False)

In [ ]:
#Check result sentence
len(predictions)

In [ ]:
#Check result data and type
predictions[0]

In [ ]:
tag_df = pd.read_csv('/kaggle/input/nithan-chadok-name-entity-recognition/tag_list.csv')

#Extract data value from dict list
final_test_df = []

for i in predictions:
    for j in i:
        for k in j.values():
            result = tag_df[tag_df['tag'] == k]['class'].values[0]
            final_test_df.append(result)

In [ ]:
#print result
print(len(final_test_df))

In [ ]:
final_result = pd.DataFrame(final_test_df)
submit_df['pred'] = final_result
submit_df

In [ ]:
submit_df[submit_df['pred'] == 0]

In [ ]:
#Save result file to CSV
submit_df.to_csv('airesearch_wangchan_16.csv',index=False)

# Baseline code

In [ ]:
import pandas as pd
from transformers import RobertaTokenizerFast, RobertaForTokenClassification, pipeline
# from attacut import tokenize
import torch
from sklearn.metrics import f1_score

In [ ]:
sample_submission = pd.read_csv('/kaggle/input/nithan-chadok-name-entity-recognition/sample_submission.csv')
df = pd.read_csv('/kaggle/input/nithan-chadok-name-entity-recognition/test.csv')
df['pred'] = sample_submission['pred']
sample_submission.head()

In [ ]:
df.head()

In [ ]:
df['end'] = 0
df['start'] = 0
all_sentences = []
sent = ''
for i, row in df.iterrows():
    all_sentences.append(str(row['word']).replace("_","[!und:]"))
    df.at[i, 'start'] = len(sent)
    sent += all_sentences[-1]
    df.at[i, 'end'] = len(sent)
    sent += ' _ '
sentence = ' _ '.join(all_sentences)

sentence[:100]

In [ ]:
tokenizer = RobertaTokenizerFast.from_pretrained("lst-nectec/HoogBERTa-NER-lst20")
model = RobertaForTokenClassification.from_pretrained("lst-nectec/HoogBERTa-NER-lst20")

In [ ]:
tag_list = pd.read_csv('/kaggle/input/nithan-chadok-name-entity-recognition/tag_list.csv')
tags = {row['tag']:row['class'] for _, row in tag_list.iterrows()}
tags

In [ ]:
nlp = pipeline('token-classification', model=model, tokenizer=tokenizer, aggregation_strategy="none")
outputs = nlp(sentence)
print(outputs)

In [ ]:
sent[:100]

In [ ]:
df.iloc[8]

In [ ]:
output_dicts = {out['start']: out for out in outputs}
output_dicts

In [ ]:
df['pred'] = 0
dummy_word = ''
for i, row in df.iterrows():
    if row['start'] in output_dicts:
        df.at[i, 'pred'] = tags[output_dicts[row['start']]['entity']]
        output_dicts.pop(row['start'])
    elif str(row['word']).isdigit():
        df.at[i, 'pred'] = tags['B_NUM']

In [ ]:
df.head()

In [ ]:
df[['i', 'pred']].to_csv('baseline.csv', index=False)